In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import spacy
import nltk
import datetime
import fasttext
import gensim.downloader as api

In [2]:
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
import json

In [4]:
with open('D:\\HWAN\\Data\\First_Aid_recommendation.json','r') as f:
    data = json.load(f)

In [5]:
df = pd.DataFrame(data['intents'])

In [6]:
df

,tag,patterns,responses,context_set
0,Cuts,"[What to do if Cuts?, How to cure Cuts?, Which...",[Wash the cut properly to prevent infection an...,
1,Abrasions,"[how do you treat abrasions?, Do Abrasions cau...",[Begin with washed hands.Gently clean the area...,
2,stings,"[How do you treat Sting?, Stings, What to do i...",[Remove any stingers immediately. Some experts...,
3,Splinter,"[How to remove Splinters, How to cure Splinter...",[1. soak it in EPSOM salts. Dissolve a cup of ...,
4,Sprains,"[How do you treat a sprain?, what to do if i g...",[Use an ice pack or ice slush bath immediately...,
5,Strains,"[How do you treat a strain?, what to do if i g...","[Rest,Ice,Compression and Elevation can be use...",
6,Fever,"[How do you treat a mild Fever?, what to do if...",[To treat a fever at home: 1)Drink plenty of f...,
7,Nasal Congestion,"[How do you treat nasal Congestion?, what to d...","[When youâ€™re stuffed up, focus on keeping yo...",
8,Cough,"[How to cure cough?, How do you treat cough?, ...","[1) Honey:- Use honey to treat a cough, mix 2 ...",
9,Sore Throat,"[How do you treat sore throat?, what to do if ...",[1) Make sure you get plenty of rest and drink...,


In [21]:
df_train = df[['tag', 'patterns']]

In [22]:
df_train

,tag,patterns
0,Cuts,"[What to do if Cuts?, How to cure Cuts?, Which..."
1,Abrasions,"[how do you treat abrasions?, Do Abrasions cau..."
2,stings,"[How do you treat Sting?, Stings, What to do i..."
3,Splinter,"[How to remove Splinters, How to cure Splinter..."
4,Sprains,"[How do you treat a sprain?, what to do if i g..."
5,Strains,"[How do you treat a strain?, what to do if i g..."
6,Fever,"[How do you treat a mild Fever?, what to do if..."
7,Nasal Congestion,"[How do you treat nasal Congestion?, what to d..."
8,Cough,"[How to cure cough?, How do you treat cough?, ..."
9,Sore Throat,"[How do you treat sore throat?, what to do if ..."


In [9]:
nlp = spacy.load('en_core_web_lg')

In [10]:
from nltk import PorterStemmer

In [11]:
stemmer = PorterStemmer()

In [31]:
def clean(sentence):
    tokens = []
    doc = nlp(sentence)
    for token in doc:
        if token.is_alpha:
            tokens.append(str(token))
    return ' '.join(tokens)
    
def stem(sentence):
    tokens = nltk.word_tokenize(sentence)
    stemmed_tokens = []
    for token in tokens:
        stemmed_tokens.append(str(stemmer.stem(token)))

    return ' '.join(stemmed_tokens)

def lemmatize(sentence):
    lemmatized_tokens = []
    doc = nlp(sentence)
    for token in doc:
        lemmatized_tokens.append(str(token.lemma_))
    
    return ' '.join(lemmatized_tokens)

In [32]:
labels = {}

for i,label in enumerate(df_train['tag']):
    labels[label] = i

In [33]:
labels

{'Cuts': 0,
 'Abrasions': 1,
 'stings': 2,
 'Splinter': 3,
 'Sprains': 4,
 'Strains': 5,
 'Fever': 6,
 'Nasal Congestion': 7,
 'Cough': 8,
 'Sore Throat': 9,
 'Gastrointestinal problems': 10,
 'Skin problems': 11,
 'Abdonominal Pain': 12,
 'Bruises': 13,
 'Broken Toe': 14,
 'Choking': 15,
 'Wound': 16,
 'Diarrhea': 17,
 'Frost bite': 18,
 'Heat Exhaustion': 19,
 'Heat Stroke': 20,
 'Insect Bites': 21,
 'nose bleed': 22,
 'Pulled Muscle': 23,
 'Rectal bleeding': 24,
 'Sun Burn': 25,
 'Testicle Pain': 26,
 'Vertigo': 27,
 'Normal Bleeding': 28,
 'Eye Injury': 29,
 'Chemical Burn': 30,
 'Poison': 31,
 'Teeth': 32,
 'seizure': 33,
 'Head Injury': 34,
 'Fainting': 35,
 'Headache': 36,
 'Cold': 37,
 'Rash': 38,
 'snake bite': 39,
 'animal bite': 40,
 'Drowning': 41,
 'CPR': 42,
 'Fracture': 43}

In [34]:
for i,patterns in enumerate(df_train['patterns']):
    Pre_proccessed_List = []
    Existing_sents_List = df_train['patterns'].loc[i]
    for sentence in Existing_sents_List:
        print(sentence)
        Pre_proccessed_List.append(lemmatize(clean(sentence)))
        Pre_proccessed_List.append(stem(clean(sentence)))
        Pre_proccessed_List.append(clean(sentence))
        Pre_proccessed_List.append(lemmatize(sentence))
        Pre_proccessed_List.append(stem(sentence))
    Existing_sents_List.extend(Pre_proccessed_List)

What to do if Cuts?
How to cure Cuts?
Which medicine to apply for Cuts?
what to apply on cuts?
Cuts
how do you treat abrasions?
Do Abrasions cause scars?
Abrasions
what to do if abrasions?
Which medicine to apply for abrasions?
How to cure abrasions?
How do you treat Sting?
Stings
What to do if you get a sting?
Which medicine to apply if sting?
How to remove Splinters
How to cure Splinters?
What to do if I have splinters?
How do you bring a splinter to the surface?
How do you treat a sprain?
what to do if i get a sprain?
Which cream to apply if i get a sprain?
Which medicine to apply if I get a sprain?
How do you treat a strain?
what to do if i get a strain?
Which cream to apply if i get a strain?
Which medicine to apply if I get a strain?
How do you diagnose a strain?
Is heat or ice better for a pulled muscle?
How do you treat a mild Fever?
what to do if i get a mild fever?
Which medicine to take if I get a mild fever?
fever
How do you treat nasal Congestion?
what to do if i get a nas

In [36]:
df_train['Numeric_label'] = df_train.tag.map(labels)

C:\Users\HP\AppData\Local\Temp\ipykernel_10380\2272514042.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['Numeric_label'] = df_train.tag.map(labels)


In [38]:
df_train = df_train[['patterns', 'Numeric_label']]

In [39]:
df_train

,patterns,Numeric_label
0,"[What to do if Cuts?, How to cure Cuts?, Which...",0
1,"[how do you treat abrasions?, Do Abrasions cau...",1
2,"[How do you treat Sting?, Stings, What to do i...",2
3,"[How to remove Splinters, How to cure Splinter...",3
4,"[How do you treat a sprain?, what to do if i g...",4
5,"[How do you treat a strain?, what to do if i g...",5
6,"[How do you treat a mild Fever?, what to do if...",6
7,"[How do you treat nasal Congestion?, what to d...",7
8,"[How to cure cough?, How do you treat cough?, ...",8
9,"[How do you treat sore throat?, what to do if ...",9


In [45]:
df_train = df_train.explode('patterns')

In [46]:
df_train.reset_index(drop=True, inplace=True)
print(df_train)

                               patterns  Numeric_label
0                   What to do if Cuts?              0
1                     How to cure Cuts?              0
2     Which medicine to apply for Cuts?              0
3                what to apply on cuts?              0
4                                  Cuts              0
...                                 ...            ...
1123             how to cure a fracture             43
1124              how to cure a fractur             43
1125             How to cure a Fracture             43
1126           how to cure a fracture ?             43
1127            how to cure a fractur ?             43

[1128 rows x 2 columns]


In [60]:
X_train = []

for i,sv in enumerate(df_train['patterns']):
    X_train.append(nlp(sv).vector)

In [61]:
X_train = np.array(X_train)

In [52]:
X_train.shape

(1128, 300)

In [62]:
X_train_reshaped = np.reshape(X_train, (X_train.shape[0], 1, -1))

In [66]:
X_train = tf.convert_to_tensor(X_train_reshaped)

In [67]:
X_train[0].shape

TensorShape([1, 300])

In [56]:
y_train = []

for i,label in enumerate(df_train['Numeric_label']):
    y_train.append(float(label))

In [57]:
y_train = tf.convert_to_tensor(np.array(y_train))

In [58]:
y_train

<tf.Tensor: shape=(1128,), dtype=float64, numpy=array([ 0.,  0.,  0., ..., 43., 43., 43.])>

In [59]:
import datetime

In [69]:
%load_ext tensorboard

In [71]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(filters=64, kernel_size=1, activation='relu', input_shape=(1, 300)),
    tf.keras.layers.GlobalMaxPooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(44, activation='softmax') 
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

logdir="logs/fit/" + datetime.time().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(X_train, y_train, epochs=100, callbacks=[tensorboard_callback])

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)                    │ (None, 1, 64)               │          19,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_max_pooling1d_2               │ (None, 64)                  │               0 │
│ (GlobalMaxPooling1D)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 44)                  │           2,860 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 26,284 (102.67 KB)

 Trainable params: 26,284 (102.67 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.0449 - loss: 4.1697
Epoch 2/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.1184 - loss: 3.4559
Epoch 3/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.1909 - loss: 3.0866
Epoch 4/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3431 - loss: 2.6184
Epoch 5/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4790 - loss: 2.1559
Epoch 6/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5942 - loss: 1.6399
Epoch 7/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6763 - loss: 1.2698
Epoch 8/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7642 - loss: 1.0621
Epoch 9/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8240 - loss: 0.7976
Epoch 10/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8608 - loss: 0.6420
Epoch 11/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8897 - loss: 0.5883
Epoch 12/100
36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accurac

In [73]:
%tensorboard --logdir logs

In [77]:
while True:
    User_input = input('User: ')
    sentence = clean(User_input)
    doc = nlp(sentence)
    vector = doc.vector
    vector = np.reshape(vector, (-1, 1, 300))
    predictions = model.predict(vector)
    predicted_class_index = np.argmax(predictions)
    predicted_class_probability = predictions[0][predicted_class_index]
    if predicted_class_probability > 0.6:
        print(predicted_class_index)
    else:
        print('I dont Understand', predicted_class_probability, predicted_class_index)

User:  how do you treat abrasions?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1


User:  How do you treat Sting?


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2


User:  "How do you handle Sting?"


...

User:  "What steps should I take if I experience a sprain?"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
I dont Understand 0.24356623 38


User:  "What should I do if I get a sprain?"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
I dont Understand 0.5503315 4


KeyboardInterrupt: Interrupted by user

In [79]:
while True:
    User_input = input('User: ')
    sentence = lemmatize(clean(User_input))
    doc = nlp(sentence)
    vector = doc.vector
    vector = np.reshape(vector, (-1, 1, 300))
    predictions = model.predict(vector)
    predicted_class_index = np.argmax(predictions)
    predicted_class_probability = predictions[0][predicted_class_index]
    if predicted_class_probability > 0.6:
        print(predicted_class_index)
    else:
        print('I dont Understand', predicted_class_probability, predicted_class_index)

User:  "What steps should I take if I experience a sprain?"


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
I dont Understand 0.4652469 26


KeyboardInterrupt: Interrupted by user

In [80]:
model.save('D:\\HWAN\\Models\\First_Aid_Recommendations\\AidSyncModel.keras')